### Import Required Libraries and Set Up Environment Variables

In [107]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [108]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [109]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"+f"&fq={filter_query}&sort={sort}&fl={field_list}")

In [110]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):  # NYT API limits results to 10 per page, 20 pages = 200 results
    

    # create query with a page number
    # API results show 10 articles at a time
    page_query = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_query).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for responses in response["response"]["docs"]:
            reviews_list.append(responses)
        # Print the page that was just retrieved
        print(f"Checked page {page}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"Page empty, no results found")
        break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [111]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [112]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_normalized_df = pd.json_normalize(reviews_list)
reviews_list_normalized_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [113]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

reviews_list_normalized_df['title'] = reviews_list_normalized_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_list_normalized_df
print(reviews_list_normalized_df['title'])

0                             The Attachment Diaries
1      What’s Love Got to Do With It?’ Probably a Lo
2                               You Can Live Forever
3                          A Tourist’s Guide to Love
4                            Other People’s Children
                           ...                      
195                                   The Other Half
196                           The Ottoman Lieutenant
197                                     Love & Taxes
198    Everybody Loves Somebody,’ a Rom-Com With Bit
199                Kedi,’ Rekindling a ‘Love of Life
Name: title, Length: 200, dtype: object


In [114]:
print(reviews_list_normalized_df.columns)

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'title'],
      dtype='object')


In [115]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_list_normalized_df["keywords"] = reviews_list_normalized_df["keywords"].apply(extract_keywords)
reviews_list_normalized_df



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody L

In [116]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_list_normalized_df["title"].squeeze().to_list()
titles_list


['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [117]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [118]:
# Create an empty list to store the results
tmdb_movies_list = []


# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1


# Loop through the titles
for title in titles_list:

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0: #and request_counter != 0:
        time.sleep(1)
        print("Sleeping for 1 seconds")
        


    # Add 1 to the request counter
    request_counter += 1

    
    # Perform a "GET" request for The Movie Database
    tmdb_response = requests.get(url + title + tmdb_key_string).json()
    #print(response)
    
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
       
        # Get movie id
        movie_id = tmdb_response["results"][0]["id"]


        # Make a request for a the full movie details
        movie_details = (f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}")
        
        # Execute "GET" request with url
        tmbd_response_details = requests.get(movie_details).json()

        # Extract the genre names into a list
        genres = [tmbd_response_details['genres'][i]['name'] for i in range(len(tmbd_response_details['genres']))]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [tmbd_response_details['spoken_languages'][i]['english_name'] for i in range(len(tmbd_response_details['spoken_languages']))]

        # Extract the production_countries' name into a list
        production_countries = [tmbd_response_details['production_countries'][i]['name'] for i in range(len(tmbd_response_details['production_countries']))]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_data = {
            "title": title,
            "original_title": tmbd_response_details['original_title'],
            "budget": tmbd_response_details['budget'],
            "original_language" : tmbd_response_details['original_language'],
            "homepage": tmbd_response_details['homepage'],
            "overview": tmbd_response_details['overview'],
            "popularity": tmbd_response_details['popularity'],
            "runtime": tmbd_response_details['runtime'],
            "release_date": tmbd_response_details['release_date'],
            "revenue": tmbd_response_details['revenue'],
            "vote_average": tmbd_response_details['vote_average'],
            "vote_count": tmbd_response_details['vote_count'],
            "genre_names": tmbd_response_details['genres'],
            "spoken_languages": tmbd_response_details['spoken_languages'],
            "production_countries": tmbd_response_details['production_countries'],
        }
        tmdb_movies_list.append(movie_data)

         # Print out success message
        print(f"Title found: {title}")

    except Exception as e:
        # Print out the title that was not found and the error
        print(f"Movie not found: {title}, error: {e}")


Title found: The Attachment Diaries
Movie not found: What’s Love Got to Do With It?’ Probably a Lo, error: list index out of range
Title found: You Can Live Forever
Title found: A Tourist’s Guide to Love
Title found: Other People’s Children
Title found: One True Loves
Title found: The Lost Weekend: A Love Story
Title found: A Thousand and One
Title found: Your Place or Mine
Title found: Love in the Time of Fentanyl
Title found: Pamela, a Love Story
Title found: In From the Side
Title found: After Love
Title found: Alcarràs
Title found: Nelly & Nadine
Title found: Lady Chatterley’s Lover
Title found: The Sound of Christmas
Title found: The Inspection
Title found: Bones and All
Title found: My Policeman
Title found: About Fate
Title found: Waiting for Bojangles
Title found: I Love My Dad
Title found: A Love Song
Title found: Alone Together
Title found: Art of Love
Title found: The Wheel
Title found: Thor: Love and Thunder
Title found: Both Sides of the Blade
Title found: Fire of Love
Tit

In [119]:
for title in titles_list:
    print(title)

The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo
You Can Live Forever
A Tourist’s Guide to Love
Other People’s Children
One True Loves
The Lost Weekend: A Love Story
A Thousand and One
Your Place or Mine
Love in the Time of Fentanyl
Pamela, a Love Story
In From the Side
After Love
Alcarràs
Nelly & Nadine
Lady Chatterley’s Lover
The Sound of Christmas
The Inspection
Bones and All
My Policeman
About Fate
Waiting for Bojangles
I Love My Dad
A Love Song
Alone Together
Art of Love
The Wheel
Thor: Love and Thunder
Both Sides of the Blade
Fire of Love
Love & Gelato
Stay Prayed Up
Benediction
Dinner in America
In a New York Minute
Anaïs in Love
I Love America
See You Then
La Mami
Love After Love
Deep Water
Lucy and Desi
Cyrano
The In Between
Book of Love
Lingui, the Sacred Bonds
The Pink Cloud
A Journal for Jordan
West Side Story
Aulcie
Love Is Love Is Love
Love Hard
Bergman Island
Hard Luck Love Song
South of Heaven
Wife of a Spy
Happier Than Ever
Together
Annette
Resort t

In [120]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))
display(tmdb_movies_list)


[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.325,
        "runtime": 102,
        "release_date": "2021-10-07",
        "revenue": 0,
        "vote_average": 3.0,
        "vote_count": 4,
        "genre_names": [
            {
                "id": 18,
                "name": "Drama"
            },
            {
                "id": 9648,
                "name": "Mystery"
            },
            {
                "id": 53,
         

[{'title': 'The Attachment Diaries',
  'original_title': 'El apego',
  'budget': 0,
  'original_language': 'es',
  'homepage': '',
  'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.',
  'popularity': 1.325,
  'runtime': 102,
  'release_date': '2021-10-07',
  'revenue': 0,
  'vote_average': 3.0,
  'vote_count': 4,
  'genre_names': [{'id': 18, 'name': 'Drama'},
   {'id': 9648, 'name': 'Mystery'},
   {'id': 53, 'name': 'Thriller'},
   {'id': 27, 'name': 'Horror'}],
  'spoken_languages': [{'english_name': 'Spanish',
    'iso_639_1': 'es',
    'name': 'Español'}],
  'production_countries': [{'iso_3166_1': 'AR', 'name': 'Argentina'}]},
 {'title'

In [121]:
# Convert the results to a DataFrame
convert_tmdb_df = pd.DataFrame(tmdb_movies_list)
convert_tmdb_df


,title,original_title,budget,original_language,homepage,overview,popularity,runtime,release_date,revenue,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.325,102,2021-10-07,0,3.000,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",58.303,96,2023-03-24,15055,6.600,31,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,A Tourist’s Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",15.245,96,2023-04-21,0,6.300,154,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,Other People’s Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",11.619,104,2022-09-21,84178,6.900,178,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'FR', 'name': 'France'}]"
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,20.397,100,2023-04-07,37820,6.500,65,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",11.999,130,2017-01-11,0,5.600,191,"[{'id': 18, 'name': 'Drama'}]","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
150,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,6.936,83,2017-01-14,0,6.000,113,"[{'id': 35, 'name': 'Comedy'}]","[{'english_name': 'French', 'iso_639_1': 'fr',...","[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is..."
151,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,4.222,103,2016-12-02,0,6.300,24,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]"
152,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,en,https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",14.827,111,2017-03-10,413844,6.170,244,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'Turkish', 'iso_639_1': 'tr'...","[{'iso_3166_1': 'TR', 'name': 'Turkey'}, {'iso..."


In [122]:
# reviews_list_normalized_df.columns
# convert_tmdb_df.columns

### Merge and Clean the Data for Export

In [123]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_list_normalized_df, convert_tmdb_df, on="title")   
merged_df.head(5)
#merged_df["title"].head(2)



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,release_date,revenue,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.325,102,2021-10-07,0,3.0,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",58.303,96,2023-03-24,15055,6.6,31,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",15.245,96,2023-04-21,0,6.3,154,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",11.619,104,2022-09-21,84178,6.9,178,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'FR', 'name': 'France'}]"
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,20.397,100,2023-04-07,37820,6.5,65,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."


In [124]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genres", "spoken_languages", "production_countries"]

# Create a list of characters to remove
columns_to_fix = merged_df.columns

# Create a list of characters to remove
chars_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    if merged_df[column].dtype == 'object':
        #pass

        # Loop through characters to remove
        for char in chars_to_remove:
            # Replace the character in the column only if it's a string
            merged_df[column] = merged_df[column].apply(lambda x: x.replace(char, "") if isinstance(x, str) else x)
    
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

# Display the fixed DataFrame
merged_df.head(2)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,release_date,revenue,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.325,102,2021-10-07,0,3.0,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",58.303,96,2023-03-24,15055,6.6,31,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."


In [125]:
print(merged_df.columns)

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'title', 'original_title', 'budget',
       'original_language', 'homepage', 'overview', 'popularity', 'runtime',
       'release_date', 'revenue', 'vote_average', 'vote_count', 'genre_names',
       'spoken_languages', 'production_countries'],
      dtype='object')


In [126]:
# Drop "byline.person" column
merged_df.drop(columns='byline.person', inplace=True)
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,release_date,revenue,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.325,102,2021-10-07,0,3.0,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",58.303,96,2023-03-24,15055,6.6,31,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourists Gui...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",15.245,96,2023-04-21,0,6.3,154,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",11.619,104,2022-09-21,84178,6.9,178,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'FR', 'name': 'France'}]"
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,20.397,100,2023-04-07,37820,6.5,65,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."


In [127]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates().reset_index(drop=True)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,release_date,revenue,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.325,102,2021-10-07,0,3.0,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",58.303,96,2023-03-24,15055,6.6,31,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourists Gui...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",15.245,96,2023-04-21,0,6.3,154,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",11.619,104,2022-09-21,84178,6.9,178,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'FR', 'name': 'France'}]"
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,20.397,100,2023-04-07,37820,6.5,65,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,...,"Miguel, a heroic Spanish doctor, puts himself ...",11.999,130,2017-01-11,0,5.6,191,"[{'id': 18, 'name': 'Drama'}]","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
150,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,...,Fiona visits Paris for the first time to assis...,6.936,83,2017-01-14,0,6.0,113,"[{'id': 35, 'name': 'Comedy'}]","[{'english_name': 'French', 'iso_639_1': 'fr',...","[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is..."
151,https://www.nyti

In [128]:
# Export data to CSV without the index
#merged_df.to_csv("output/NYT_TMDB.csv", index=False, encoding='utf-8-sig')
merged_df.to_csv("output/NYT_TMDB.csv", index=False)
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,release_date,revenue,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.325,102,2021-10-07,0,3.0,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",58.303,96,2023-03-24,15055,6.6,31,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourists Gui...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",15.245,96,2023-04-21,0,6.3,154,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",11.619,104,2022-09-21,84178,6.9,178,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'FR', 'name': 'France'}]"
4,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,20.397,100,2023-04-07,37820,6.5,65,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."
